In [1]:
import pandas as pd
from CPM import CPMRegressor
import joblib
import numpy as np

In [2]:
# from 1_original_version.ipynb
indices = joblib.load('indices')
all_masks = joblib.load('all_masks')
subject_list = joblib.load('sub_list_noNA')
behav_obs_pred = joblib.load('behav_obs_pred').map(float)

# train the new model
y = pd.read_csv('../../../data/age.csv', index_col=0)
X = pd.read_pickle('../../../data/movie.dataframe')

In [3]:
for fold in range(3):
    test_this_subs = [x for i, x in enumerate(subject_list) if indices[i] == fold]
    train_this_subs = [x for i, x in enumerate(subject_list) if indices[i] != fold]

    y_test = y.loc[test_this_subs]
    X_test = X.loc[test_this_subs]
    y_train = y.loc[train_this_subs]
    X_train = X.loc[train_this_subs]

    model = CPMRegressor(threshold=0.05)
    model.fit(X_train, y_train.values.flat)

    new_test = y_test

    old_df = behav_obs_pred.query('`test fold` == @fold')
    new_df = model.predict(X_test)

    print(
        '#----------------------------------',
        f'fold-{fold}: '
        f'positive mask is all same? : {all(all_masks["pos"][fold] == model.mask["positive"])}',
        f'fold-{fold}: '
        f'negative mask is all same? : {all(all_masks["neg"][fold] == model.mask["negative"])}',

        f'fold-{fold}: '
        f'positive predict is all same? : '
        f'r = {np.corrcoef(new_df["positive"], old_df["emotion predicted (pos)"])[0, 1]:.4f}  '
        f'new = {np.mean(new_df["positive"]):.2f}, old = {np.mean(old_df["emotion predicted (pos)"]):.2f}',

        f'fold-{fold}: '
        f'negative predict is all same? : '
        f'r = {np.corrcoef(new_df["negative"], old_df["emotion predicted (neg)"])[0, 1]:.4f}   '
        f'new = {np.mean(new_df["negative"]):.2f}, old = {np.mean(old_df["emotion predicted (neg)"]):.2f}',

        f'fold-{fold}: '
        f'GLM predict is all same? : '
        f'r = {np.corrcoef(new_df["both"], old_df["emotion predicted (glm)"])[0, 1]:.4f}  '
        f'new = {np.mean(new_df["both"]):.2f}, old = {np.mean(old_df["emotion predicted (glm)"]):.2f}',

        sep='\n'
    )

#----------------------------------
fold-0: positive mask is all same? : True
fold-0: negative mask is all same? : True
fold-0: positive predict is all same? : r = 1.0000  new = 10.96, old = 10.96
fold-0: negative predict is all same? : r = 1.0000   new = 10.90, old = 10.90
fold-0: GLM predict is all same? : r = 1.0000  new = 11.08, old = 11.08
#----------------------------------
fold-1: positive mask is all same? : True
fold-1: negative mask is all same? : True
fold-1: positive predict is all same? : r = 1.0000  new = 10.99, old = 10.99
fold-1: negative predict is all same? : r = 1.0000   new = 10.98, old = 10.98
fold-1: GLM predict is all same? : r = 1.0000  new = 10.86, old = 10.86
#----------------------------------
fold-2: positive mask is all same? : True
fold-2: negative mask is all same? : True
fold-2: positive predict is all same? : r = 1.0000  new = 10.93, old = 10.93
fold-2: negative predict is all same? : r = 1.0000   new = 11.01, old = 11.01
fold-2: GLM predict is all same